In [5]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import curdoc

def my_tap_handler(attr,old,new):
    index = source.selected.indices
    print(index)

source = ColumnDataSource(data=dict(
    x=[[1,2,3],[7,8,8,]],
    y=[[2,1,3],[6,8,7]]
))
p = figure(tools="tap")

renderer = p.patches('x', 'y', source=source)
renderer.data_source.on_change("selected", my_tap_handler)
curdoc().add_root(p)
show(p)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [32]:
import numpy as np
import pandas as pd
from bokeh.models.callbacks import CustomJS
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource, CustomJSFilter, TapTool, HoverTool


source = ColumnDataSource(pd.DataFrame(
    np.array([[i, i] for i in range(0, 10)]), columns=['x', 'y']))

data = [[i, "%s.%d" % ("data", i)] for i in range(0, 10)]
data.extend([[i, "%s.%d" % ("data", i)] for i in range(0, 10)])
source2 = ColumnDataSource(pd.DataFrame(data, columns = ['x', 'data']))

columns = [
        TableColumn(field='x', title='x'),
        TableColumn(field='data', title='data'),
    ]
data_table = DataTable(source=source2, columns=columns, width=600, height=400)

selection_callback = CustomJS(args=dict(source=source, source2=source2, p2=data_table), code="""
    var indices = []

    var source_x = source.data.x[source.selected.indices[0]]
    source2.data.x.forEach((source2_x, i) => {
        if (source2_x == source_x) {
          indices.push(i)
        }
    });

    source2.selected.indices = indices
    p2.change.emit()
""")

plot = figure(title="Plot1", tools="tap,pan,box_zoom,wheel_zoom,save,reset")
plot.select(TapTool).callback = selection_callback
source.selected.js_on_change('indices', selection_callback)
plot.circle('x', 'y', source=source, line_color=None, color='red', size=6)

show(row(plot, data_table))

In [16]:

import sys
sys.path.append('..')
from bokeh.io import show, output_notebook
from bokeh.plotting import gmap, figure, show, curdoc
from bokeh.events import ButtonClick, Event, Tap
from bokeh.models import *
from bokeh.layouts import column, row
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from AI.Data.database_handler import get_all_data, get_n_data
# from HelperFunctions import *
# import constant

from random import random
# from bokeh.layouts import row 
# from bokeh.models import ColumnDataSource, CustomJS
# from bokeh.plotting import figure, show 
# from bokeh.events import ButtonClick


lines = []
def ConsolePrint(input_string):
    if (len(lines)>40):
        lines.pop(0)
    line = input_string
    lines.append(line)
    text = "<br>".join(lines)
    div.text = text

def ClearConsole(event):
    lines.clear()
    lines.append("Console cleared...")
    text = "<br>".join(lines)
    div.text = text


# text_input = TextInput(title="Console", value=" ")
div = Div(text="", width=200, max_height=500, style={'overflow-y': 'hidden'})

jab = Button(label="Just a Button", button_type="primary", sizing_mode='stretch_height')


x = [random() for x in range(1000)]
y = [random() for y in range(1000)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width = 500, plot_height = 500, tools="lasso_select", title="Make Your Selection Here")
p1.circle('x', 'y', source=s1, alpha=0.7)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=500, plot_height=500, x_range=(0,1), y_range=(0,1),
tools="save", title="Look Over Here")
p2.circle('x','y', source=s2, alpha=0.7)

## Callback
s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1,s2=s2),
code="""
console.log(s2.data)
var inds = cb_obj.indices;
console.log(inds);
var d1 = s1.data;
var d2 = s2.data;
d2['x'] = []
d2['y'] = []

for (var i = 0; i < inds.length; i++){
	d2['x'].push(d1['x'][inds[i]])
	d2['y'].push(d1['y'][inds[i]])
}
console.log(s2.data)

s2.change.emit(); // push data to python 
"""
))


# layout = column(row(p1,p2), row(jab,div))

# layout = column(row(p1,p2), row(text_input, div))

# p2.on_change('value', ConsolePrint)
# jab.on_change(ButtonClick, ClearConsole)

show(row(p1,p2))
# show(layout)


In [19]:
# Import Stuffs
from bokeh import events
from bokeh.plotting import figure, output_file, show, save
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models import HoverTool, TapTool
from bokeh.models import ColumnDataSource, CustomJS, Div, Rect
from bokeh.events import Tap
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from IPython.display import IFrame
from IPython.core.display import display, HTML
import tempfile
import numpy as np

In [20]:
# Deepnote Show function for Bokhe
def bokeh_deepnote_show(plot):
    tmp_output_filename = tempfile.NamedTemporaryFile(suffix='.html').name
    output_file(tmp_output_filename)
    save(plot)

    f = open(tmp_output_filename, "r")
    display(HTML(f.read()))

In [31]:
source = ColumnDataSource(dict(
    x=[1, 2, 2, 1, 2, 3],
    y=[1, 1, 2, 2, 3, 2],
    indicies=[0,1,2,3,4,5]
))

fig = figure(title='My Coordinates: Click Anywhere to Mark',
             plot_height=500, plot_width=600,
             x_range=(0, 4), y_range=(0, 4))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='green', size=10, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('x', '@x'), ('y', '@y'), ('index', '$index'), ('indicies', '@indicies')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

callback = CustomJS(args=dict(source=source), code="""
    // change data source from Callback
    var mainData = source.data;
    //var inds = cb_obj;
    //console.log(inds)
    //console.log(cb_obj);
    //console.log(cb_data);
    var inds = mainData['indicies'];
    mainData['x'].push(cb_obj.x);
    mainData['y'].push(cb_obj.y);
    mainData['indicies'].push(inds[inds.length-1]+1);
    console.log(mainData);
    source.change.emit();
""")

fig.js_on_event(Tap, callback)

args=dict(source=source)
print(args)

bokeh_deepnote_show(fig)

{'source': ColumnDataSource(id='5082', ...)}


In [8]:
def display_event(div, attributes=[]):
    """
    Function to build a suitable CustomJS to display the current event
    in the div model.
    """
    style = 'float: left; clear: left; font-size: 13px'
    return CustomJS(args=dict(div=div), code="""
        const attrs = %s;
        const args = [];
        for (let i = 0; i < attrs.length; i++) {
            const val = JSON.stringify(cb_obj[attrs[i]], function(key, val) {
                return val.toFixed ? Number(val.toFixed(2)) : val;
            })
            args.push(attrs[i] + '=' + val)
        }
        const line = "<span style=%r><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        const text = div.text.concat(line);
        const lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

In [30]:
source = ColumnDataSource(dict(
    x=[1, 2, 2, 1, 2, 3],
    y=[1, 1, 2, 2, 3, 2],
))

selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='My Coordinates: Select the Circles',
             plot_height=500, plot_width=600,
             x_range=(0, 4), y_range=(0, 4))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='green', size=10, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('x', '@x'), ('y', '@y')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())

current_indices = []

callback = CustomJS(args=dict(source=source, selected=selected, current_indices = current_indices), code="""
    // get data source from Callback args
    console.log(source.data);
    let data = Object.assign({}, source.data);
    let indices = Object.assign([], source.selected.indices);
    data.x = indices.map(i => data.x[i]);
    data.y = indices.map(i => data.y[i]);
    selected.data = data;
    current_indices = source.selected.indices
    console.log(selected.data);
    console.log(current_indices);
    selected.change.emit();
""")


callback2 = CustomJS(args=dict(source=source, current_indices = current_indices), code="""
    console.log(source.data);
    //console.log(cb_obj);
    //console.log(cb_data);
    let indices = Object.assign([], source.selected.indices);
    current_indices = source.selected.indices
    console.log(current_indices);
""")

fig.js_on_event(Tap, callback2)

bokeh_deepnote_show(fig)

# print(current_indices)
print(selected.data)

{'x': [], 'y': []}


In [27]:
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.plotting import figure, show 

# Data
x = [2,3,5,6,9,7]
y = [6,5,3,8,7,5]

# Network graph
edges = {
	0: [1,2],
	1: [0,3,4],
	2: [0,5],
	3: [1,4],
	4: [1,3],
	5: [2,3,4]
}

# Create figure
p = figure(plot_width=500, plot_height=500, tools="",toolbar_location=None, title="Hove over nodes")

source = ColumnDataSource(dict(x0 = [], y0=[], x1=[],y1=[] ))

sr = p.segment(x0='x0',y0='y0',x1='x1',y1='y1', color="firebrick", alpha=0.5, line_width=2, source=source)
cr = p.circle(x, y, color="firebrick", size=30, alpha=0.4, hover_color="navy", hover_alpha=1.0)

js_code = """
const edges = %s;
const data = {'x0':[], 'y0':[], 'x1':[], 'y1':[]};
const ind = cb_data.index.indices;
for (var i = 0; i < ind.length; i++){
	const start = ind[i]
	for (var j = 0; j < edges[start].length; j++){
		const end = edges[start][j]
		data['x0'].push(circle.data.x[start])
		data['y0'].push(circle.data.y[start])
		data['x1'].push(circle.data.x[end])
		data['y1'].push(circle.data.y[end])
	}
}
segment.data = data
""" % edges

cb = CustomJS(args={'circle':cr.data_source, 'segment':sr.data_source},code=js_code)
p.add_tools(HoverTool(tooltips=None, callback=cb, renderers=[cr]))

show(p)

In [6]:
from bokeh.plotting import figure
from bokeh.client import push_session
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, CustomJS, BoxSelectTool, Range1d, Rect
source = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))
callback = CustomJS(args=dict(source=source), code="""
        var data = source.get('data');
        var geometry = cb_data['geometry'];
        var width = geometry['x1'] - geometry['x0'];
        var height = geometry['y1'] - geometry['y0'];
        var x = geometry['x0'] + width/2;
        var y = geometry['y0'] + height/2;
        data['x'].push(x);
        data['y'].push(y);
        data['width'].push(width);
        data['height'].push(height);
        source.trigger('change');
    """)
box_select = BoxSelectTool(callback=callback)
p = figure(plot_width=400,
           plot_height=400,
           tools=[box_select],
           title="Select Below",
           x_range=Range1d(start=0.0, end=1.0),
           y_range=Range1d(start=0.0, end=1.0))
rect = Rect(x='x',
            y='y',
            width='width',
            height='height',
            fill_alpha=0.3,
            fill_color='#009933')
p.add_glyph(source, rect, selection_glyph=rect, nonselection_glyph=rect, name="selectionbox")
session = push_session(curdoc())
def toolEventsCallback(attr, old, new):
    print("callback", new)
    x0 = new[0]['x0']
    x1 = new[0]['x1']
    print("x0=%f  x1=%f" % (x0, x1))
p.tool_events.on_change("geometries", toolEventsCallback)
session.show() 
session.loop_until_closed() 

ValueError: js_on_change takes an event name and one or more callbacks, got only one parameter

In [12]:
source = ColumnDataSource(dict(
    x=[1, 2, 2, 1, 2, 3],
    y=[1, 1, 2, 2, 3, 2],
))

selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='My Coordinates: Select the Circles',
             plot_height=500, plot_width=600,
             x_range=(0, 4), y_range=(0, 4))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='green', size=10, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('x', '@x'), ('y', '@y')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())

callback = CustomJS(args=dict(source=source, selected=selected), code="""
    // get data source from Callback args
    let data = Object.assign({}, source.data);
    let indices = Object.assign([], source.selected.indices);
    data.x = indices.map(i => data.x[i]);
    data.y = indices.map(i => data.y[i]);
    selected.data = data;
    console.log(selected.data);
    selected.change.emit();
""")

source2 = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))
callback2 = CustomJS(args=dict(source=source2), code="""
        var data = source.get('data');
        var geometry = cb_data['geometry'];
        var width = geometry['x1'] - geometry['x0'];
        var height = geometry['y1'] - geometry['y0'];
        var x = geometry['x0'] + width/2;
        var y = geometry['y0'] + height/2;
        data['x'].push(x);
        data['y'].push(y);
        data['width'].push(width);
        data['height'].push(height);
        source.trigger('change');
    """)

rect = Rect(x='x',
            y='y',
            width='width',
            height='height',
            fill_alpha=0.3,
            fill_color='#009933')
fig.add_glyph(source, rect, selection_glyph=rect, nonselection_glyph=rect, name="selectionbox")

fig.js_on_event(Tap, callback)

bokeh_deepnote_show(fig)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "height" value "height", key "width" value "width" [renderer: GlyphRenderer(id='2142', ...)]


In [9]:
from bokeh.plotting import ColumnDataSource
# define ColumnDataSource
source = ColumnDataSource(
    data=dict(
        x=[1, 2, 3, 4, 5],
        y=[2, 5, 8, 2, 7],
        desc=['A', 'b', 'C', 'd', 'E'],
    )
)
# find max for variable 'x' from 'source'
print( max( source.data['x'] ))

5


In [ ]:

def display_event(div, attributes=[]):
    """
    Function to build a suitable CustomJS to display the current event
    in the div model.
    """
    style = 'float: left; clear: left; font-size: 13px'
    return CustomJS(args=dict(div=div), code="""
        const attrs = %s;
        const args = [];
        for (let i = 0; i < attrs.length; i++) {
            const val = JSON.stringify(cb_obj[attrs[i]], function(key, val) {
                return val.toFixed ? Number(val.toFixed(2)) : val;
            })
            args.push(attrs[i] + '=' + val)
        }
        const line = "<span style=%r><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        const text = div.text.concat(line);
        const lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))



def CreateGrid(grid_size):
    grid_indices = []
    for i in range(grid_size*grid_size):
        grid_indices.append(i)
    
    source = ColumnDataSource(dict(
        x=CalculateGrid(grid_size, grid_lower_lat, grid_lower_lon, grid_upper_lat, grid_upper_lon)[0],
        y=CalculateGrid(grid_size, grid_lower_lat, grid_lower_lon, grid_upper_lat, grid_upper_lon)[1],
        indicies=grid_indices
    ))
    
    grid = p.rect(source=source, x='x', y='y',
        alpha=0.5, color='red', fill_color=None, line_width=1, 
        width=FindSize(grid_size)[0], height=FindSize(grid_size)[1],
        hover_line_color='black', hover_alpha=0.7, hover_line_width=3)
    
    tooltips = [('x', '@x'), ('y', '@y'), ('index', '$index')]
    p.add_tools(HoverTool(tooltips=tooltips, renderers=[grid]))
    
    return grid, source




def InsertText(attrname, old, new):
    line = new
    lines.append(line)
    text= "<br>".join(lines)
    div.text = text
    text_input.value=""
    print(grid_indices.data)
    print(grid_indices.data["index"])